In [1]:
from acousticnn.main_dir import main_dir
import os
import seaborn as sns
os.chdir(main_dir)

%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from acousticnn.plate.dataset import HDF5Dataset
from acousticnn.utils.argparser import get_args, get_config
import torch
from torch.utils.data import ConcatDataset

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

difficulty = "G5000_no_sampling"
args = get_args(["--config", f"cfg/{difficulty}.yaml", "--model_cfg", "fqo_rn18.yaml"])
config = get_config(args.config)
config.dataset_keys = ["bead_patterns", "z_vel_mean_sq", "phy_para", "frequencies"]
G5000_dataset  = HDF5Dataset(args, config, config.data_path_train, normalization=True)
difficulty = "V5000_no_sampling"
args = get_args(["--config", f"cfg/{difficulty}.yaml", "--model_cfg", "fqo_rn18.yaml"])
config = get_config(args.config)
config.dataset_keys = ["bead_patterns", "z_vel_mean_sq", "phy_para", "frequencies"]
V5000_dataset  = HDF5Dataset(args, config, config.data_path_train, normalization=True)
dataset = ConcatDataset([G5000_dataset, V5000_dataset])

In [42]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 5})
figsize = (1.45, 1)
rcParams["figure.figsize"] = figsize
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.Set2(np.linspace(0,1,2)))
save_dir = "plots/dataset_figure/"

In [43]:
from scipy.signal import find_peaks

def detect_peaks(dataset, prominence=.5, wlen=100):
    peaks_list = []
    pixel_sums = []
    prominence_list = []
    for i in range(len(dataset)):
        img, response = dataset[i]["bead_patterns"], dataset[i]["z_vel_mean_sq"]
        pixel_sum  = torch.sum(img > 0) / (img.shape[1] * img.shape[2])
        pixel_sums.append(pixel_sum)
        actual_peaks, properties = find_peaks(response, prominence=prominence, wlen=wlen)
        prominence_list.append(properties["prominences"])
        peaks_list.append(actual_peaks)
    len_peaks = np.array([len(peak) for peak in peaks_list])
    pixel_sums = np.array(pixel_sums)
    peaks = np.hstack(peaks_list)
    return len_peaks, pixel_sums, peaks,  np.hstack(prominence_list), peaks_list

In [ ]:
def do_boxplot(len_peaks, pixel_sums, peaks):
    data_dict = {}
    for len_val in set(len_peaks):
        data_dict[len_val] = pixel_sums[len_peaks == len_val]

    data = [data_dict[len_val] for len_val in sorted(data_dict.keys())]
    fig, ax = plt.subplots()
    boxplot = ax.boxplot(data, showfliers=False, patch_artist=True, vert=False)

    box_color = 'lightblue'
    whisker_color = 'gray'
    for patch in boxplot['boxes']:
        patch.set_facecolor(box_color)
        patch.set_edgecolor('black')
    for whisker in boxplot['whiskers']:
        whisker.set(color=whisker_color, linewidth=1.5, linestyle='--')
    for median in boxplot['medians']:
        median.set(color='red', linewidth=2)
    ax.set_yticklabels(sorted(data_dict.keys()))
    ax.tick_params(axis='both', which='major')
    plt.xlabel('Ratio of pixels with beads')
    plt.ylabel('Peak count per image')
    plt.tight_layout()
    return fig

len_peaks, pixel_sums, peaks, prominences, peaks_list = detect_peaks(dataset)
fig = do_boxplot(len_peaks, pixel_sums, peaks)
sns.despine(offset=5)

plt.tight_layout()
#plt.savefig(save_dir + "peak_count_beadratio.svg", format='svg', dpi = 600)
plt.show()

fig, ax = plt.subplots()
ax.hist(peaks, bins=np.linspace(-0.5, 300.5, 30), density=False,color='steelblue', edgecolor='black',linewidth=0.5, stacked=True)
plt.xlabel('Frequency')
plt.ylabel("Peak count")
ax.set_xticks([0, 100, 200, 300])

sns.despine(offset=5)
plt.tight_layout()
plt.savefig(save_dir + "peak_count_frequency.svg", format='svg', dpi = 600, transparent=True)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(1.5, 2))
all_patterns = torch.stack([torch.nn.functional.interpolate(dataset[i]["bead_patterns"].unsqueeze(0), size=(81, 121), \
                                                            mode='bilinear', align_corners=True)[0] for i in range(len(dataset))])
ax[0].imshow(torch.mean(all_patterns, axis=0)[0], cmap=plt.cm.gray)

ax[0].axis('off')  # Turn off the axis labels
freqs = np.linspace(-1, 1, 300)
out_mean1, out_mean2 = dataset.datasets[0].out_mean, dataset.datasets[1].out_mean
out_std1, out_std2 = dataset.datasets[0].out_std, dataset.datasets[1].out_std
ax[1].plot((out_mean1[freqs] + out_mean2[freqs]) / 2, lw=1)
ax[1].set_ylim(-25, 75)
sns.despine(offset=5)
ax[1].set_xticks([0, 100, 200, 300])
ax[1].set_yticks([-25, 0, 25, 50, 75])
ax[1].grid(which="major", lw=0.2)
plt.xlabel('Frequency')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.savefig(save_dir + "dataset_mean.svg", format='svg', dpi = 600, transparent=True)


In [ ]:
len_peaks, pixel_sums, peaks, prominences, peaks_list = detect_peaks(V5000_dataset)
print(np.argmax(len_peaks), np.max(len_peaks))
print(np.argmin(len_peaks), np.min(len_peaks))
i, i2 = np.argmax(len_peaks), np.argmin(len_peaks)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(1.5*1.5,  1.2*1.5)) # , gridspec_kw={'height_ratios': [1, 0.5]}
bead_pattern1, response1 = V5000_dataset[i]["bead_patterns"][0], V5000_dataset[i]["z_vel_mean_sq"]
response1 = (response1 * out_std1 + out_mean1[freqs])
bead_pattern2, response2 = V5000_dataset[i2]["bead_patterns"][0], V5000_dataset[i2]["z_vel_mean_sq"]
response2 = (response2 * out_std1 + out_mean1[freqs])

ax[0].axis('off')  # Turn off the axis labels
plt.subplot(2, 2, 1)  # Create a subplot grid with 2 rows and 2 columns, and select the first subplot
plt.imshow(bead_pattern1, cmap=plt.cm.gray)
plt.axis('off')  # Turn off the axis labels

plt.subplot(2, 2, 2)  # Select the second subplot
plt.imshow(bead_pattern2, cmap=plt.cm.gray)
plt.axis('off')  # Turn off the axis labels


plt.subplot(2, 1, 2)  # Select the second subplot
plt.plot(response1, label="left plate", lw=1)
plt.plot(response2, label="right plate", lw=1)
ax[1].set_ylim(-25, 75)
ax[1].set_xticks([0, 100, 200, 300])
ax[1].set_yticks([-25, 0, 25, 50, 75])
ax[1].grid(which="major", lw=0.2)
sns.despine(offset=5)

plt.plot(peaks_list[i] - 1, response1[peaks_list[i]], 'x', markersize=6, color="r")
plt.plot(peaks_list[i2], response2[peaks_list[i2]], 'x', markersize=6, color="r")
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.legend()

plt.subplots_adjust(hspace=0.2)
plt.tight_layout()
plt.savefig(save_dir + "two_single_examples.svg", format='svg', dpi = 600, transparent=True)

## compare different datasets

In [ ]:
len_peaks1, pixel_sums1, peaks1, prominence1, peaks_list1 = detect_peaks(V5000_dataset)
len_peaks2, pixel_sums2, peaks2, prominence2, peaks_list2 = detect_peaks(G5000_dataset)

data = [pixel_sums1, pixel_sums2]
data = [len_peaks1, len_peaks2]

# Plotting the boxplots
fig, ax = plt.subplots(figsize=(1.43, 0.86))
boxplot = ax.boxplot(data, showfliers=False, patch_artist=True, vert=False)
box_color = 'lightblue'
whisker_color = 'gray'
for patch in boxplot['boxes']:
    patch.set_facecolor(box_color)
    patch.set_edgecolor('black')
for whisker in boxplot['whiskers']:
    whisker.set(color=whisker_color, linewidth=0.5, linestyle='--')
for median in boxplot['medians']:
    median.set(color='red', linewidth=1)
plt.xlabel('Number of peaks')
yticklabels = ['V-5000', "G-5000"]
plt.yticks(range(1, len(yticklabels) + 1), yticklabels)
plt.xticks([0,2, 4, 6, 8, 10, 12])
plt.tick_params(axis='y')
sns.despine(offset=5)
plt.grid(which="major", lw=0.2)
plt.tight_layout()
plt.savefig(save_dir + "dataset_comparison.svg", format='svg', dpi = 600, transparent=True)
plt.show()
